In [1]:
import numpy as np
import time
import glob, os, sys
import pickle
from subprocess import call

import numpy as np
from keras import backend as K
from keras.preprocessing import image
import keras
from keras.layers import Flatten, Dropout, Dense

from keras.applications.resnet50 import ResNet50 as resnet
from keras.applications.resnet50 import preprocess_input as resnet_pp

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
IMG_SIZE = 224

In [3]:
cnn_object, pp_function, img_size = resnet, resnet_pp, IMG_SIZE
cnn_model = cnn_object(weights='imagenet', include_top=False, input_shape=(IMG_SIZE,IMG_SIZE,3))
# base_model.add(keras.layers.Lambda(pp_function, name='preprocessing', input_shape=(img_size, img_size, 3)))

/home/mikey/.local/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [4]:
base_model = keras.models.Sequential()

base_model.add(cnn_model)

# first: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = False
    
# model = base_model
base_model.add(keras.layers.InputLayer(input_shape=(1,1,2048)))
base_model.add(Flatten(input_shape=(1,1,2048)))
base_model.add(Dropout(0.10))
base_model.add(Dense(units=1024, activation='relu'))
base_model.add(Dropout(0.25))
base_model.add(Dense(units=10, activation='softmax'))

# compile the model (should be done *after* setting layers to non-trainable)
opt = keras.optimizers.Adam(lr=0.0001)
base_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
top_layers_checkpoint_path = 'cp.h5'
train_data_dir = './train/'
nb_train_samples = 6270
nb_validation_samples = 6270
top_epochs = 400
batch_size = 32

if os.path.exists(top_layers_checkpoint_path):
    base_model.load_weights(top_layers_checkpoint_path)
    print ("Checkpoint '" + top_layers_checkpoint_path + "' loaded.")

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(preprocessing_function=resnet_pp,
                                   rotation_range=45,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(preprocessing_function=resnet_pp)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    train_data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size,
    class_mode='categorical')

#Save the model after every epoch.
mc_top = ModelCheckpoint(top_layers_checkpoint_path,
                         monitor='val_acc', 
                         verbose=0,
                         save_best_only=True, 
                         save_weights_only=False, 
                         mode='auto', 
                         period=1)

Checkpoint 'cp.h5' loaded.
Found 6270 images belonging to 10 classes.
Found 6270 images belonging to 10 classes.


In [9]:
base_model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples // batch_size,
    nb_epoch=top_epochs,
    validation_data=validation_generator,
#     nb_val_samples=nb_validation_samples // batch_size,
    nb_val_samples=200,
    callbacks=[mc_top]
)

/home/mikey/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/home/mikey/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=<keras_pre..., callbacks=[<keras.ca..., steps_per_epoch=6, epochs=400, validation_steps=200)`
  


Epoch 1/400
6/6 [==============================] - 26s 4s/step - loss: 4.8102 - acc: 0.5833 - val_loss: 3.4716 - val_acc: 0.6954
Epoch 2/400
5/6 [========================>.....] - ETA: 0s - loss: 4.3089 - acc: 0.6188

KeyboardInterrupt: 